In [1]:
import pandas as pd
import numpy as np
from pandasai.llm.openai import OpenAI
from pandasai import SmartDataframe
from dotenv import load_dotenv
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
import plotly.express as px

In [2]:
# Load environment variables from .env (ensure you have OpenAI keys set here)
load_dotenv()

# Load your CSV data into a pandas DataFrame
df = pd.read_csv("basededados_aulat7.csv")

# Initialize OpenAI LLM for PandasAI
llm = OpenAI(temperature=0)

# Use PandasAI's SmartDataframe for natural language queries on the dataframe
sdf = SmartDataframe(df, config={"llm": llm})

# Initialize LangChain's OpenAI model for broader language tasks
chat_openai = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Create a LangChain agent for Pandas DataFrame using the loaded DataFrame and LLM
agent = create_pandas_dataframe_agent(chat_openai, df, verbose=True, allow_dangerous_code=True)


C:\Users\PedroLustosa\AppData\Local\Temp\ipykernel_14412\3150843877.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  chat_openai = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")


In [7]:
# Combined function to process the question and return filtered data, plot, and insights
def process_question(question: str):
    # Step 1: Filter DataFrame using SmartDataframe based on question
    print("Filtering data based on question...")
    filtered_df = sdf.chat(question)

    # Check if filtered_df is a DataFrame
    if isinstance(filtered_df, pd.DataFrame):
        info_ = extract_info(filtered_df)
        if info_ == "empty":
            return "No data available for the given query.", None
        elif info_ == "text":
            return filtered_df, provide_insights(question)
        else:
            chart = choose_visualization(filtered_df)
            if chart == "No recommendation available":
                return filtered_df, provide_insights(question)
            else:
                plot = create_plot(filtered_df, chart)
                insights = provide_insights(question)
                return filtered_df, plot, insights
    else:
        return "Error: The filtered data is not a DataFrame.", None

def extract_info(df: pd.DataFrame) -> str:
    """Determine the type of information contained in the DataFrame."""
    if not isinstance(df, pd.DataFrame):
        return "error"
    if df.empty:
        return "empty"
    elif df.shape[0] == 1:
        return "text"
    else:
        return "chart"

def extract_metadata(df: pd.DataFrame) -> dict:
    """Extract metadata from the DataFrame."""
    try:
        metadata = {
            'Number of Columns': df.shape[1],
            'Schema': {col: str(df[col].dtype) for col in df.columns},
            'Data Types': df.dtypes.apply(lambda x: str(x)).to_dict(),
            'Sample': df.head(1).to_dict(orient="records")
        }
    except Exception as e:
        print(f"Error extracting metadata: {e}")
        metadata = {"Error": "Could not extract metadata"}
    return metadata

def choose_visualization(df: pd.DataFrame) -> str:
    """Choose the best chart type based on DataFrame metadata."""
    metadata = extract_metadata(df)
    metadata_message = "\n".join(f"{key}: {value}" for key, value in metadata.items())
    prompt = f"Given the following DataFrame description, recommend the best visualization:\n{metadata_message}"
    
    try:
        response = llm(prompt)
        recommendation = response['choices'][0]['text'].strip()
        print(f"Recommended visualization: {recommendation}")
    except Exception as e:
        print(f"Error generating recommendation: {e}")
        recommendation = "No recommendation available"
    
    return recommendation.lower()

def create_plot(df: pd.DataFrame, chart_type: str):
    """Create a plot based on the recommended chart type."""
    try:
        if chart_type == "line":
            fig = px.line(df)
        elif chart_type == "bar":
            fig = px.bar(df)
        elif chart_type == "scatter":
            fig = px.scatter(df)
        else:
            fig = px.histogram(df)
        return fig
    except Exception as e:
        print(f"Error creating plot: {e}")
        return None

def provide_insights(query: str):
    """Provide insights from the DataFrame based on a natural language query."""
    try:
        response = agent.run(query)
        return response
    except Exception as e:
        print(f"Error providing insights: {e}")
        return "Could not provide insights"

def cast_to_datetime(df: pd.DataFrame) -> pd.DataFrame:
    """Convert columns with date information to datetime."""
    for col in df.columns:
        if 'data' in col.lower() or 'date' in col.lower():
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
                print(f"Column '{col}' has been converted to datetime.")
            except Exception as e:
                print(f"Could not convert column '{col}' to datetime: {e}")
    return df

In [8]:
# Example usage: Input a natural language question
question = "Top 5 fornecedores?"

# Process the question to get filtered data, plot, and insights
filtered_data, insights = process_question(question)

# Display filtered data and insights
print("Filtered Data:")
print(filtered_data)

print("\nInsights:")
print(insights)



Filtering data based on question...
                                      fornecedor  qtd
81      PROCTER & GAMBLE IND COM. LTDA(ED/ENX/F)  739
52       KIMBERLY-CLARK KENKO INDUST (FEMININOS)  377
76  PROCTER & GAMBLE DO BRASIL S/A. (ABS PREMIO)  342
80        PROCTER & GAMBLE HIG E COSM LTDA (C.D)  312
87        PROCTER & GAMBLE IND E COML LTDA(AMAC)  299
Error generating recommendation: 'OpenAI' object is not callable
Error creating plot: Plotly Express cannot process wide-form data with columns of different type.
Thought: To find the top 5 fornecedores, we need to count the occurrences of each fornecedor in the dataframe.
Action: python_repl_ast
Action Input: df['fornecedor'].value_counts().head(5)PROCTER & GAMBLE DO BRASIL S/A(PANTENE)         99
CIMEX DITRIBUIDORA DE COSMETICOS LTDA           92
PROCTER & GAMBLE IND E COML LTDA(AMAC)          54
PROCTER & GAMBLE DO BRASIL S/A. (ABS PREMIO)    51
PROCTER & GAMBLE IND COM. LTDA(ED/ENX/F)        48
Name: fornecedor, dtype: int64I no

ValueError: too many values to unpack (expected 2)

In [9]:
filtered_data